Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
With a convolutional neural network that trains to 100% accuracy on these images

In [0]:
import os
import zipfile
from urllib.request import urlretrieve

# Download the dataset from Google API

urlretrieve("https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip", 
            filename="/tmp/happy-or-sad.zip")

# Unzip the dataset to following directory

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()


In [19]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

def train_happy_sad_model():

  DESIRED_ACCURACY = 0.999
  
  
    # Stop training when hitting %99.9 accuracy
    
  class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if(logs.get('acc') > DESIRED_ACCURACY):
        print('\n\nReached %99.9 accuracy so cancelling training!\n\n')
        self.model.stop_training = True

  callbacks = myCallback()
    
    
    # This Code Block Define and Compile the Model
    
  model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation=tf.nn.relu, input_shape=(150,150,3)),
        tf.keras.layers.MaxPooling2D(2, 2),    
        tf.keras.layers.Conv2D(32, (3,3), activation=tf.nn.relu),
        tf.keras.layers.MaxPooling2D(2, 2),    
        tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu),
        tf.keras.layers.MaxPooling2D(2, 2),    
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation=tf.nn.relu),
        tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)  
    ]) 

  model.compile(loss='binary_crossentropy', optimizer = RMSprop(lr=0.001), metrics=['acc'])
        
    # This code block create an instance of an ImageDataGenerator called train_datagen 
    # And a train_generator by calling train_datagen.flow_from_directory

  train_datagen = ImageDataGenerator(rescale = 1/255)

  train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s', 
        target_size = (150, 150), 
        batch_size = 128,
        class_mode = 'binary'
    )

    # This code block call model.fit_generator and train for
  
    # model fitting    
    
  history = model.fit_generator(
      train_generator,
      steps_per_epoch=8,  
      epochs=10,
      verbose=1,
      callbacks = [callbacks])
    
  return history.history['acc'][-1]
  
train_happy_sad_model()

W0821 20:24:38.659364 139804613187456 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0821 20:24:38.953551 139804613187456 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Found 80 images belonging to 2 classes.
Epoch 1/10
8/8 [==============================] - 15s 2s/step - loss: 1.7645 - acc: 0.5000
Epoch 2/10
8/8 [==============================] - 13s 2s/step - loss: 0.4678 - acc: 0.7109
Epoch 3/10
8/8 [==============================] - 13s 2s/step - loss: 0.2139 - acc: 0.9297
Epoch 4/10
8/8 [==============================] - 13s 2s/step - loss: 0.0685 - acc: 0.9766
Epoch 5/10
7/8 [=========================>....] - ETA: 1s - loss: 0.0295 - acc: 1.0000

Reached %99.9 accuracy so cancelling training!


8/8 [==============================] - 13s 2s/step - loss: 0.0283 - acc: 1.0000


1.0